In [140]:
import pandas as pd
import numpy as np
with open('weather.txt') as f:
    data = f.readlines()

data2010 = []
    
for line in data:
    if "2010" in line[0:21]:
        data2010.append(line)
    
print(data2010[0])


MX000017004201001TMAX-9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999   -9999     278  S-9999   



In [141]:
alldata = []
i = 0
for month in data2010:
    if "PRCP" not in month:
        essentials = ["MX17004",2010,month[15:17],month[17:21]]
        mondata = month[21:].split()
        alldata.append(essentials + mondata)
        i += 1
    if i == 10:
        break
alldata2 = [list(i) for i in zip(*alldata)]
df = pd.DataFrame(alldata2).T


df.columns = ["id","year","month","element","day1","day2","day3","day4","day5","day6","day7",
                                "day8","day9","day10","day11","day12","day13","day14","day15","day16","day17","day18","day19","day20",
                                "day21","day22","day23","day24","day25","day26","day27","day28","day29","day30","day31"]

df = df.replace(to_replace=["-9999","S-9999","S"],value="--")
df = pd.melt(df,["id","year","month","element"])
df = df[df.value !="--"]
df = df.replace(to_replace=["day"],value="",regex=True)
df = df.assign(date = df.year.astype(str) + "-" + df.month.astype(str) + "-" + df.variable.astype(str)[:] )
df.pop("year")
df.pop("month")
df.pop("variable")
df = df[["id","date","element","value"]]
print(df)

          id        date element value
12   MX17004   2010-02-2    TMAX   273
13   MX17004   2010-02-2    TMIN   144
32   MX17004   2010-02-4    TMAX   241
33   MX17004   2010-02-4    TMIN   144
44   MX17004   2010-03-5    TMAX   321
45   MX17004   2010-03-5    TMIN   142
94   MX17004  2010-03-10    TMAX   345
95   MX17004  2010-03-10    TMIN   168
112  MX17004  2010-02-12    TMAX   297
113  MX17004  2010-02-12    TMIN   134
154  MX17004  2010-03-16    TMAX   311
155  MX17004  2010-03-16    TMIN   176
232  MX17004  2010-02-24    TMAX   299
233  MX17004  2010-02-24    TMIN   107
266  MX17004  2010-04-27    TMAX   363
267  MX17004  2010-04-27    TMIN   167
268  MX17004  2010-05-27    TMAX   332
269  MX17004  2010-05-27    TMIN   182
290  MX17004  2010-01-30    TMAX   278
291  MX17004  2010-01-30    TMIN   145


In [144]:
dfk = df.pivot(["id","date"],"element","value")
print(dfk)

element            TMAX TMIN
id      date                
MX17004 2010-01-30  278  145
        2010-02-12  297  134
        2010-02-2   273  144
        2010-02-24  299  107
        2010-02-4   241  144
        2010-03-10  345  168
        2010-03-16  311  176
        2010-03-5   321  142
        2010-04-27  363  167
        2010-05-27  332  182
